In [4]:
import os
from glob import glob
from monai.transforms import (
   Activations,
   #AsChannelFirstd,
   AsDiscrete,
   CenterSpatialCropd,
   Compose,
   LoadImaged,
   MapTransform,
   NormalizeIntensityd,
   Orientationd,
   RandFlipd,
   RandScaleIntensityd,
   RandShiftIntensityd,
   RandSpatialCropd,
   Spacingd,
   ToTensord,
)

/gpu_home/bori/miniconda3/envs/sam_adapt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

data_path='/gpu_home/bori/nnUNetFrame/dataset'
data_path = os.path.join(data_path, "nnUNet_raw", "Dataset001_Postoperative")
image_path =sorted(glob(os.path.join(data_path, "imagesTr", '*.nii.gz')))
label_path =sorted(glob (os.path.join(data_path, "labelsTr", '*.nii.gz')))

train_images =sorted(glob(os.path.join(data_path, "imagesTr", '*.nii.gz')))
train_labels =sorted(glob (os.path.join(data_path, "labelsTr", '*.nii.gz')))
train_labels
train_files = [{"image": image_name, "label":label_name} for image_name, label_name in zip(train_images, train_labels)]
len(train_files)

20

In [6]:
train_files[:3]

[{'image': '/gpu_home/bori/nnUNetFrame/dataset/nnUNet_raw/Dataset001_Postoperative/imagesTr/Image_Recidief10_0000.nii.gz',
  'label': '/gpu_home/bori/nnUNetFrame/dataset/nnUNet_raw/Dataset001_Postoperative/labelsTr/Image_Recidief1.nii.gz'},
 {'image': '/gpu_home/bori/nnUNetFrame/dataset/nnUNet_raw/Dataset001_Postoperative/imagesTr/Image_Recidief11_0000.nii.gz',
  'label': '/gpu_home/bori/nnUNetFrame/dataset/nnUNet_raw/Dataset001_Postoperative/labelsTr/Image_Recidief10.nii.gz'},
 {'image': '/gpu_home/bori/nnUNetFrame/dataset/nnUNet_raw/Dataset001_Postoperative/imagesTr/Image_Recidief12_0000.nii.gz',
  'label': '/gpu_home/bori/nnUNetFrame/dataset/nnUNet_raw/Dataset001_Postoperative/labelsTr/Image_Recidief11.nii.gz'}]

In [ ]:

roi_size=[128, 128, 64]
pixdim=(1.5, 1.5, 2.0)

train_transform = Compose(
   [
       # load 4 Nifti images and stack them together
       LoadImaged(keys=["image", "label"]),
       #ConvertToMultiChannelBasedOnBratsClassesd(keys="label"),
       Spacingd(
           keys=["image", "label"],
           pixdim=pixdim,
           mode=("bilinear", "nearest"),
       ),
       Orientationd(keys=["image", "label"], axcodes="RAS"),
       RandSpatialCropd(
           keys=["image", "label"], roi_size=roi_size, random_size=False),
       RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=0),
       NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True),
       RandScaleIntensityd(keys="image", factors=0.1, prob=0.5),
       RandShiftIntensityd(keys="image", offsets=0.1, prob=0.5),
       ToTensord(keys=["image", "label"])
   ]
)
